In [1]:
# import _config_mac_to_aws as _config
import _config

In [2]:
from go.bot.go_db import GoDB
from sqlmodel import Session, SQLModel, create_engine
from go.bot.playfab_api import PlayfabApi, as_player_id, as_playfab_id
from go.bot.playfab_db import PlayfabDB
from go.bot.models import GoTeam
from pprint import pprint

engine = create_engine(_config.godb_url, echo=_config.godb_echo)

SQLModel.metadata.create_all(engine)
pfdb = PlayfabDB()
godb = GoDB()

pfapi = PlayfabApi()
# pfapi.login_to_playfab()



# Run Some Queries

In [ ]:
from sqlmodel import select
from go.bot.models import GoRatings,PfPlayer
from go.bot.playfab_api import as_playfab_id


session = Session(engine)
statement = select(GoRatings, PfPlayer).where(GoRatings.rating_type == "official").where(GoRatings.season == "GO Phoenix S1")
statement = statement.where(GoRatings.pf_player_id == PfPlayer.id)
statement = statement.order_by(GoRatings.go_rating.desc())
ratings = session.exec(statement).all()

# print(len(ratings))
for r in ratings:
    # print(r)
    print(f"{r[1].ign}\t{as_playfab_id(r[1].id)}\t{r[0].go_rating}")

# Choose a playfabid

In [3]:
stooobe_playfabid = '121559C6C39206B5'
lester_playfabid = 'B6395EBFD2459486'
alexfromc_playfabid = 'F9F21325130975C'
revolutionx_playfabid = '4575D3B047A8981E'
coffee_backon_playfabid = as_playfab_id(1065470778045992877)
bradwolf_playfabid = as_playfab_id(-119964107432152410)
provolown_pfid = '62D75D8FC9785191'
munii_playfabid = as_playfab_id(5135846081841770710)

pfid = munii_playfabid

In [4]:
print(pfapi.get_player_from_account_info(player_id=as_player_id(pfid)))
print("stats:")
print(pfapi.get_player_career_stats(player_id=as_player_id(pfid)))

Player[GO_STOOOBE, id -7920325659441625419, created 2021-05-26, last_login 2024-06-26 02:19]
stats:
date=datetime.datetime(2024, 7, 1, 14, 8, 44, 563021) pf_player_id=-7920325659441625419 games=8333 wins=4021 kills=31712 damage=6621000 mmr=4900 skill=2342


In [8]:
session = Session(engine)
Stoobe = godb.read_player(discord_id=408731638223208448, session=session)
godb.create_team('solo stooobe', [Stoobe], session, rating_limit=None, season=_config.go_season)

2024-06-14 13:48:34.119  INFO      go.bot.go_db   -- Reading GoPlayer with discord_id = 408731638223208448 from DB
2024-06-14 13:48:34.221  INFO      go.bot.go_db   -- Creating GoTeam team_name = 'solo stooobe' in DB
2024-06-14 13:48:34.221  INFO      go.bot.go_db   -- Reading GoTeam with discord_ids = {408731638223208448} from DB
2024-06-14 13:48:34.307  INFO      go.bot.go_db   -- Returning team with team.id = 70


GoDbError: Team with roster {'GO_STOOOBE'} already exists.

# Add an official GoRating

In [5]:
from go.bot.models import GoRatings

with Session(engine) as session:

    player = pfdb.read_player(pf_player_id=as_player_id(pfid), session=session)
    print(f'{player =}')
    if player is None:
        raise Exception()

    career_stats = [_ for _ in player.career_stats]
    career_stats.sort(key=lambda x: x.date)
    pprint(career_stats)

    stats = career_stats[-1]
    print(f'{stats =}, rating={stats.calc_rating()}')

    go_rating = godb.get_official_rating(player.id, session, season=_config.go_season)
    print(f'{go_rating =}')
    
    if go_rating is None:
        official_rating = GoRatings(
            pf_player_id=player.id,
            season=_config.go_season,
            rating_type="official",
            go_rating=stats.calc_rating(),
            )
        session.add(official_rating)
        session.commit()
        
        print(f'{official_rating =}')


2024-07-09 16:28:52.748  INFO      go.bot.playfab_db -- Reading PfPlayer with Playfab ID 5135846081841770710 from DB


player =PfPlayer(ign='GO_MuniiBunii', account_created=datetime.datetime(2022, 7, 18, 18, 59, 13), avatar_url=None, id=5135846081841770710, last_login=datetime.datetime(2024, 6, 25, 0, 51, 38))
[PfCareerStats(date=datetime.datetime(2024, 2, 22, 0, 8, 12), wins=4409, kills=20474, mmr=4870, pf_player_id=5135846081841770710, games=14744, damage=5322016, skill=2271),
 PfCareerStats(date=datetime.datetime(2024, 4, 3, 1, 1, 55), wins=4845, kills=23063, mmr=5000, pf_player_id=5135846081841770710, games=15788, damage=5952597, skill=2741),
 PfCareerStats(date=datetime.datetime(2024, 5, 2, 16, 27, 7), wins=5159, kills=24880, mmr=4507, pf_player_id=5135846081841770710, games=16580, damage=6417573, skill=2561),
 PfCareerStats(date=datetime.datetime(2024, 6, 5, 21, 18, 54), wins=5395, kills=26343, mmr=4367, pf_player_id=5135846081841770710, games=17137, damage=6791157, skill=1964),
 PfCareerStats(date=datetime.datetime(2024, 7, 1, 14, 13, 53), wins=5478, kills=27722, mmr=4774, pf_player_id=513584608

# Register with Playfab 
Ran once at beginning of project

In [ ]:
# headers = {
#     'Content-Type': 'application/json',
#     #'X-SecretKey': api_key, 
#     # 'X-Authorization': session_ticket
# }

# payload = {
#     "TitleId": title_id,
#     "Username": "devstooobe",  # Replace with the desired username
#     "Email": "brianstube@gmail.com",  # Replace with the user's email
#     "Password": "vabiewu2398fh"  # Replace with the user's password
# }

# response = run_request('RegisterPlayFabUser', payload, headers)